**Import**

In [1]:
import gym
from gym import wrappers
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
import random
import collections
from torch.nn import init

**Gym Wrappers**

In [2]:
import numpy as np
from collections import deque
import gym
from gym import spaces
import cv2

class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.randint(1, self.noop_max + 1) #pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done  = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert somtimes we stay in lives == 0 condtion for a few frames
            # so its important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype='uint8')
        self._skip       = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

class ClipRewardEnv(gym.RewardWrapper):
    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)

class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env):
        """Warp frames to 84x84 as done in the Nature paper and later work."""
        gym.ObservationWrapper.__init__(self, env)
        self.width = 84
        self.height = 84
        self.observation_space = spaces.Box(low=0, high=255, shape=(self.height, self.width, 1), dtype=np.uint8)

    def observation(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        return frame[:, :, None]

class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[0], shp[1], shp[2] * k), dtype=np.uint8)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self.get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self.get_ob(), reward, done, info

    def get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))

class ScaledFloatFrame(gym.ObservationWrapper):
    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0

class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.
        This object should only be converted to numpy array before being passed to the model.
        You'd not believe how complex the previous solution was."""
        self._frames = frames

    def __array__(self, dtype=None):
        out = np.concatenate(self._frames, axis=2)
        if dtype is not None:
            out = out.astype(dtype)
        return out

class ImageToPyTorch(gym.ObservationWrapper):
    """
    Change image shape to CWH
    """
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], old_shape[0], old_shape[1]), dtype=np.uint8)

    def observation(self, observation):
        return np.swapaxes(observation, 2, 0)

def make_atari(env_id):
    env = gym.make(env_id)
    assert 'NoFrameskip' in env.spec.id
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=4)
    return env

def wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=False, scale=False, pytorch_img=False):
    """Configure environment for DeepMind-style Atari.
    """
    if episode_life:
        env = EpisodicLifeEnv(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = WarpFrame(env)
    if scale:
        env = ScaledFloatFrame(env)
    if clip_rewards:
        env = ClipRewardEnv(env)
    if frame_stack:
        env = FrameStack(env, 4)
    if pytorch_img:
        env = ImageToPyTorch(env)
    return env

**Policy**

In [3]:
class DQN(nn.Module):

  def __init__(self, num_actions):
    super(DQN, self).__init__()
    # Same structure as the Human Control DQN in 2015
    self.conv = nn.Sequential(nn.Conv2d(4, 32, kernel_size=8, stride=4), nn.ReLU(),
                              nn.Conv2d(32, 64, kernel_size=4, stride=2), nn.ReLU(),
                              nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.ReLU())
    self.fc = nn.Sequential(nn.Linear(7 * 7 * 64, 512), nn.ReLU(), nn.Linear(512, num_actions))
  
  def forward(self, input):
    x = self.conv(input)
    output = self.fc(x.view(-1, 7 * 7 * 64))
    return output

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
        
class RNDModel(nn.Module):
    def __init__(self):
        super(RNDModel, self).__init__()


        feature_output = 7 * 7 * 64
        self.predictor = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=32,
                kernel_size=8,
                stride=4),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=4,
                stride=2),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1),
            nn.LeakyReLU(),
            Flatten(),
            nn.Linear(feature_output, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512)
        )

        self.target = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=32,
                kernel_size=8,
                stride=4),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=4,
                stride=2),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1),
            nn.LeakyReLU(),
            Flatten(),
            nn.Linear(feature_output, 512)
        )

        for p in self.modules():
            if isinstance(p, nn.Conv2d):
                init.orthogonal_(p.weight, np.sqrt(2))
                p.bias.data.zero_()

            if isinstance(p, nn.Linear):
                init.orthogonal_(p.weight, np.sqrt(2))
                p.bias.data.zero_()

        for param in self.target.parameters():
            param.requires_grad = False

    def forward(self, next_obs):
        target_feature = self.target(next_obs)
        predict_feature = self.predictor(next_obs)

        return predict_feature, target_feature

In [4]:
class RunningMeanStd(object):
    # https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Parallel_algorithm
    def __init__(self, epsilon=1e-4, shape=()):
        self.mean = np.zeros(shape, 'float64')
        self.var = np.ones(shape, 'float64')
        self.count = epsilon

    def update(self, x):
        batch_mean = np.mean(x, axis=0)
        batch_var = np.var(x, axis=0)
        batch_count = x.shape[0]
        self.update_from_moments(batch_mean, batch_var, batch_count)

    def update_from_moments(self, batch_mean, batch_var, batch_count):
        delta = batch_mean - self.mean
        tot_count = self.count + batch_count

        new_mean = self.mean + delta * batch_count / tot_count
        m_a = self.var * (self.count)
        m_b = batch_var * (batch_count)
        M2 = m_a + m_b + np.square(delta) * self.count * batch_count / (self.count + batch_count)
        new_var = M2 / (self.count + batch_count)

        new_count = batch_count + self.count

        self.mean = new_mean
        self.var = new_var
        self.count = new_count

**Replay Buffer**

In [5]:
class ReplayBuffer(object):
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.buffer.append([state, action, reward, next_state, done])
    
    def sample(self, batch_size):
        state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return state, action, reward, next_state, done
    
    def __len__(self):
        return len(self.buffer)

class CuriosityReplayBuffer(object):
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, int_reward, next_state, done):
        self.buffer.append([state, action, reward, int_reward, next_state, done])
    
    def sample(self, batch_size):
        state, action, reward, int_reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return state, action, reward, int_reward, next_state, done
    
    def __len__(self):
        return len(self.buffer)

**Hyper Parameters**

In [6]:
torch.manual_seed(1)
np.random.seed(1)
lr = 0.00025
alpha = 0.95
batch_size = 64
target_update = 5000
gamma = 0.99
gamma2 = 0.995
rep_buf_size = 30000
rep_buf_ini = 10000
skip_frame = 4
max_frame = 3e7
epsilon_start = 1
epsilon_final = 0.01
epsilon_decay = 250000
rnd_update_epochs = 1
rnd_lr = 0.0001
init_max_frame = 10000
int_coef = 0.001
new_DQN = True

**Set Up Env and Model**

In [7]:
env = make_atari('VentureNoFrameskip-v4')


env = wrap_deepmind(env, clip_rewards=False, frame_stack=True, pytorch_img=True, episode_life=True)
device   = torch.device("cuda")
model = DQN(env.action_space.n).to(device)
target_model = DQN(env.action_space.n).to(device)
optimizer = torch.optim.RMSprop(model.parameters(), lr=lr, alpha=alpha)
replay_buffer = CuriosityReplayBuffer(rep_buf_size)
int_reward_ms = RunningMeanStd()
observation_ms = RunningMeanStd(shape=(1, 1, 84, 84))
rnd_model = RNDModel().to(device)
rnd_optimizer = torch.optim.Adam(rnd_model.parameters(), lr=rnd_lr)

**Helper Functions**

In [8]:
def huber_loss(input, target, beta=1, size_average=True):
    """
    very similar to the smooth_l1_loss from pytorch, but with
    the extra beta parameter
    """
    n = torch.abs(input - target)
    cond = n < beta
    loss = torch.where(cond, 0.5 * n ** 2 / beta, n - 0.5 * beta)
    if size_average:
        return loss.mean(), (input - target).mean()
    return loss.sum(), (input - target)

epsilon_by_frame = lambda step_idx: epsilon_final + (epsilon_start - epsilon_final) * math.exp(-1. * step_idx / epsilon_decay)

def get_episodic_curiosity(curiosity_list):
  ret = list()
  # Reverse the list
  reversed_cur_list = curiosity_list[::-1]
  max_cur = -100
  discount_factor = gamma2
  curr_cur = max_cur
  # Loop through the list
  for curiosity in reversed_cur_list:
    if(max_cur < curiosity):
      ret.append(curiosity)
      discount_factor = gamma2
      curr_cur = curiosity
      max_cur = curiosity
    else:
      curr_cur = curr_cur  * discount_factor
      ret.append(curr_cur)
      discount_factor *= gamma2
  return np.array(ret[::-1])



**Initialize the Normalizer of Observations**

In [9]:
def normalize_observation(observation):
  return ((observation - observation_ms.mean) / np.sqrt(observation_ms.var)).clip(-5, 5)

init_frame = 0

rnd_observations = list()
while init_frame < init_max_frame:
    observation = env.reset()
    done = False
    while not done:
      action = random.sample(range(env.action_space.n), 1)[0]
      next_observation, _, done, _ = env.step(action)
      rnd_observations.append(observation[-1])
      observation = next_observation
      init_frame += 1
observation_ms.update(np.array(rnd_observations))

KeyboardInterrupt: ignored

**Train**

In [11]:
episode = 0
score = 0
frame = 0
int_score = 0
while frame < max_frame:
  observation = env.reset()
  done = False
  q_losses = 0
  i = 0
  episode_buffer = list()
  while not done:
        i += 1
        with torch.no_grad():
          t_observation = torch.from_numpy(observation).float().to(device) / 255
          t_observation = t_observation.view(1, 4, 84,84)
          epsilon = epsilon_by_frame(frame)
          if random.random() > epsilon:
            q_value = model(t_observation)
            action = q_value.argmax(1).data.cpu().numpy().astype(int)[0]
          else:
            action = random.sample(range(env.action_space.n), 1)[0]
          next_observation, reward, done, info = env.step(action)
          frame += 1
          score += reward
          reward = np.sign(reward)
          episode_buffer.append([observation, action, reward, next_observation, done])
          observation = next_observation

        # Update policy
        if len(replay_buffer) > rep_buf_ini and frame % skip_frame == 0:
            observations, actions, rewards, int_rewards, next_observations, dones = replay_buffer.sample(batch_size)          

            observations = torch.from_numpy(np.array(observations) / 255).float().to(device)
            
            actions = torch.from_numpy(np.array(actions).astype(int)).float().to(device)
            actions = actions.view(actions.shape[0], 1)
            
            rewards = torch.from_numpy(np.array(rewards)).float().to(device)
            rewards = rewards.view(rewards.shape[0], 1)

            int_rewards = torch.from_numpy(np.array(int_rewards)).float().to(device)
            int_rewards = int_rewards.view(int_rewards.shape[0], 1)
  

            next_observations = torch.from_numpy(np.array(next_observations) / 255).float().to(device)
            
            dones = torch.from_numpy(np.array(dones).astype(int)).float().to(device)
            dones = dones.view(dones.shape[0], 1)
            
            q_values = model(observations)
            next_q_values = target_model(next_observations)

            q_value = q_values.gather(1, actions.long())
            next_q_value = next_q_values.max(1)[0].unsqueeze(1)
            expected_q_value = (rewards + int_rewards * int_coef) + gamma * next_q_value * (1 - dones)

            loss, q_loss = huber_loss(q_value, expected_q_value)
            q_losses += q_loss
            optimizer.zero_grad()
            loss.backward()
            
            optimizer.step()
            
        if frame % target_update == 0:
            target_model.load_state_dict(model.state_dict())
            # Update replay buffer
        if(len(episode_buffer) > 10000):
          break

  int_rewards = list()
  rnd_observations = list()
  # Obtain the intrinsic_rewards before normalization
  for observation, action, reward, next_observation, done in episode_buffer:
    pred, target = rnd_model(torch.FloatTensor(normalize_observation(next_observation[-1])).to(device))
    int_reward = torch.pow(pred-target, 2).sum().cpu().detach().numpy()
    int_rewards.append(int_reward)
    rnd_observations.append(next_observation[-1])

  # Update the normalizer of rewards
  int_reward_ms.update(np.array(int_rewards))

  # Normalize the intrinsic rewards
  normalized_int_rewards = ((int_rewards - int_reward_ms.mean) / np.sqrt(int_reward_ms.var)).clip(-10, 10)
  if(new_DQN):
    # Lifelong intrinisic rewards
    life_long_int_rewards =  (normalized_int_rewards + 1).clip(1, 5)
    #int_score += int_rewards.mean()
    episodic_int_rewards = get_episodic_curiosity(normalized_int_rewards).clip(0, 10)
    int_rewards = life_long_int_rewards * episodic_int_rewards
  else:
    int_rewards = normalized_int_rewards.clip(0, 10)
  int_score += int_rewards.mean()
  #print(episodic_int_rewards.max(), life_long_int_rewards.max(), int_rewards.max())
  # Append transition tuples to the replay buffer
  for i in range(len(episode_buffer)):
    observation, action, reward, next_observation, done= episode_buffer[i]
    int_reward = int_rewards[i]
    replay_buffer.push(observation, action, reward, int_reward, next_observation, done)

  # Update observations
  observation_ms.update(np.array(rnd_observations))
  rnd_observations = torch.FloatTensor(normalize_observation(rnd_observations)).to(device).view(-1, 1, 84, 84)
  for i in range(rnd_update_epochs):
      pred, target = rnd_model(rnd_observations)
      rnd_loss = 0.5 * torch.pow(pred-target, 2).mean()
      rnd_optimizer.zero_grad()
      rnd_loss.backward()
      rnd_optimizer.step()

  if info['ale.lives'] == 0:
    episode += 1
    print('episode', episode, 'frame', frame, 'score', score, 'curiosity', int_score)
    #with open(reward_path, "a") as text_file:
      #text_file.write(str(score) + ' ' + str(frame) + '\n')
    score = 0
    int_score = 0

episode 1 frame 266 score 75.0 curiosity 0.6930406464370287
episode 2 frame 599 score 0.0 curiosity 0.884237611928377
episode 3 frame 1752 score 1625.0 curiosity 0.6394771526346454


KeyboardInterrupt: ignored